In [65]:
# import own 共用 lib code
import sys
sys.path.append(r"../data/data_checking_code/")

import folder_tool
import tool
import vis_tool

import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

import cv2
import tensorflow.keras as keras
import shutil
import numpy as np
import matplotlib.pyplot as plt
import glob

from tqdm.notebook import tqdm

In [66]:
# check output folder
for img_type in ['train', 'val', 'test']:
    folder_tool.check_folder('./result_post_process/{}_pred_raw'.format(img_type))
folder_tool.check_folder('./result_post_process/seg_raw/')

In [67]:
# Input
# TODO 使用 CEJ 圖片
# CEJ_images_folder = '' # run all images in ./result/
CEJ_images_folder = './result/seg_raw/'
o_img_folder = '../data/1_o_image_512/'
assert len(glob.glob(o_img_folder + '/*.PNG')) > 0, 'wrong o_img_path'
if CEJ_images_folder:
    CEJ_img_paths = glob.glob(CEJ_images_folder + '/*.PNG')
else:
    CEJ_img_paths = glob.glob('./result/*_raw/*.PNG')
# TODO input 預期是 img_size*img_size 的圖片
img_size = 512
black_img = np.zeros((img_size, img_size))

In [68]:
tar_img_id = [
    # "NN_110919_161350_C04CBC",
    # "NN_100825_103844_C0381C",
    # "NN_100825_103850_C0381F",
    # "NN_120827_095402_BFFF3E",
    # "NN_170619_152245_16F2F5",
    # "NN_171005_134750_98B6EE",
    # "NN_180122_144804_16F0BD",
    # "NN_180717_140845_98BC97",
    # "NN_180717_140916_98BC9B"
]

In [69]:
for img_path in tqdm(CEJ_img_paths):
    # get cej point
    if len(tar_img_id) != 0 and \
            os.path.basename(img_path)[:-4] not in tar_img_id:
        continue
    # print(os.path.basename(img_path))

    CEJ_mask = cv2.imread(img_path, 0)

    pixel_y_cord_list = np.where(CEJ_mask == 255)[0]
    # vis_tool.dist_vis(pixel_y_cord_list, 5, os.path.basename(img_path), 20)


    contours, _ = cv2.findContours(CEJ_mask.astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key = lambda x: np.mean(np.transpose(x)[0]))
    CEJ_link_mask = black_img.copy()

    # link the contours
    ex_cont_pt = []
    for cont in contours:
        cv2.drawContours(CEJ_link_mask, [cont], -1, 255, cv2.FILLED)
        start_pt = min(cont, key = lambda x: (x[0][0], x[0][1]))[0]
        end_pt = max(cont, key = lambda x: (x[0][0], x[0][1]))[0]
        # left most point
        if len(ex_cont_pt) > 0:
            cv2.line(CEJ_link_mask, ex_cont_pt, start_pt, 255, 1)
        ex_cont_pt = end_pt
    target_path = img_path.replace('./result/', './result_post_process/')
    o_img = cv2.imread('../data/1_o_image_512/' + os.path.basename(img_path), 0)
    
    # vis_tool.visualize([CEJ_mask, CEJ_link_mask, o_img])
    
    cv2.imwrite(target_path, CEJ_link_mask)
    # CEJ_link_test = cv2.imread(target_path, 0)
    # assert len(CEJ_link_test.shape) == 2, CEJ_link_test.shape
    # assert (np.unique(CEJ_link_test) == np.unique(CEJ_mask)).all(), np.unique(CEJ_link_test)


  0%|          | 0/533 [00:00<?, ?it/s]